<a href="https://colab.research.google.com/github/ScarlettQiu/OCR_for_Bank_Statement/blob/main/OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import openai

openai.api_key = 'YOUR KEY'

In [ ]:
# imports
import ast  # for converting embeddings saved as strings back to arrays
import openai  # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
from scipy import spatial  # for calculating vector similarities for search


# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

In [ ]:
# text copied and pasted from: https://en.wikipedia.org/wiki/Curling_at_the_2022_Winter_Olympics
# I didn't bother to format or clean the text, but GPT will still understand it
# the entire article is too long for gpt-3.5-turbo, so I only included the top few sections

wikipedia_article_on_curling = """Page|lof3|
Platinum Card | Platinum Mastercard ending in 8508
Billing|Cycle

[Aug|O3,|2022|-|Sep|02,2022|  | |31|days|

Account Summary

Payment Information

Previous Balance
Payments
Other Credits

Transactions

Payment Due Date

sep 27, 2022

New Balance Minimum Payment Due

$1,659.63 $25.00

LATE PAYMENT WARNING: If we do not receive your minimum payment
by your due date, you may have to pay a late fee of up to $40.00.

For online and phone payments, the
deadline is 8pm ET.

Cash Advances
Fees Charged
Interest Charged
New Balance

MINIMUM PAYMENT WARNING:!f you make only the minimum
payment each period, you will pay more in interest and it will take you
longer to pay off your balance. For example:

Credit Limit

Available Credit (as of Sep 02, 2022)

Cash Advance Credit Limit

Available Credit tor Cash Advances

Estimated savings if Dalance is paid off in about 3 years: $1,378

Manage your account
if you would like information about credil counseling services, call 1-888-326-8055.

$948.58

- $250.00
$0.00

+ $961.05
+ $0.00

+ $0.00

+ $0.00

= $1,659.63
$2,000.00
$340.37
$1,000.00
$340.37

anywhere, anytime.

Pay your bill, set up alerts and more

with the Capital One* mobile app.

Notifications
Welcome|to your|account|notifications,|Check||back|here|each|month| for |important|updates |about|your|account,

Pay|or|manage |your account|at|capitalone.com|

cans



Discover new features with

LAS|VEGAS, |NV/891/10-3437













Sep|27,|2022 Account|ending|in|8508
$25.00 City of| industry CA[91

and only one check (or one money order) payable to Capital One ta DAL ete dea dene eed tana eda aba
to ensure your payment |is processed|promptly, Allow at least seven|business days for delivery.

the Capital One Mobile app.



1 5/1/780/5/7/5/068/7/8)/508 165963020000002)/500/5

"""

In [ ]:
query = f"""Use the below article on the Platinum Mastercard ending in 8508 Billing Cycle. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{wikipedia_article_on_curling}
\"\"\"

Question: Which is the minimun payment due?"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the Platinum Mastercard ending in 8508 Billing Cycle.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

The minimum payment due is $25.00.


In [ ]:
query = f"""Use the below article on the Platinum Mastercard ending in 8508 Billing Cycle. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{wikipedia_article_on_curling}
\"\"\"

Question: Which is the interest charged?"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the Platinum Mastercard ending in 8508 Billing Cycle.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

Based on the information provided in the article, the interest charged for the Platinum Mastercard ending in 8508 Billing Cycle is $0.00.


In [ ]:
query = f"""Use the below article on the Platinum Mastercard ending in 8508 Billing Cycle. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{wikipedia_article_on_curling}
\"\"\"

Question: How much is the payments?"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the Platinum Mastercard ending in 8508 Billing Cycle.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

The payments made on the Platinum Mastercard ending in 8508 Billing Cycle are $250.00.


In [ ]:
# text copied and pasted from: https://en.wikipedia.org/wiki/Curling_at_the_2022_Winter_Olympics
# I didn't bother to format or clean the text, but GPT will still understand it
# the entire article is too long for gpt-3.5-turbo, so I only included the top few sections

cc1 = """Page|lof3|
Platinum Card | Platinum Mastercard ending in 8508
Billing|Cycle

[Aug|O3,|2022|-|Sep|02,2022|  | |31|days|

Account Summary

Payment Information

Previous Balance
Payments
Other Credits

Transactions

Payment Due Date

sep 27, 2022

New Balance Minimum Payment Due

$1,659.63 $25.00

LATE PAYMENT WARNING: If we do not receive your minimum payment
by your due date, you may have to pay a late fee of up to $40.00.

For online and phone payments, the
deadline is 8pm ET.

Cash Advances
Fees Charged
Interest Charged
New Balance

MINIMUM PAYMENT WARNING:!f you make only the minimum
payment each period, you will pay more in interest and it will take you
longer to pay off your balance. For example:

Credit Limit

Available Credit (as of Sep 02, 2022)

Cash Advance Credit Limit

Available Credit tor Cash Advances

Estimated savings if Dalance is paid off in about 3 years: $1,378

Manage your account
if you would like information about credil counseling services, call 1-888-326-8055.

$948.58

- $250.00
$0.00

+ $961.05
+ $0.00

+ $0.00

+ $0.00

= $1,659.63
$2,000.00
$340.37
$1,000.00
$340.37

anywhere, anytime.

Pay your bill, set up alerts and more

with the Capital One* mobile app.

Notifications
Welcome|to your|account|notifications,|Check||back|here|each|month| for |important|updates |about|your|account,

Pay|or|manage |your account|at|capitalone.com|

cans



Discover new features with

LAS|VEGAS, |NV/891/10-3437













Sep|27,|2022 Account|ending|in|8508
$25.00 City of| industry CA[91

and only one check (or one money order) payable to Capital One ta DAL ete dea dene eed tana eda aba
to ensure your payment |is processed|promptly, Allow at least seven|business days for delivery.

the Capital One Mobile app.



1 5/1/780/5/7/5/068/7/8)/508 165963020000002)/500/5

"""

In [ ]:
query = f"""Use the below article on the Platinum Mastercard ending in 8508 Billing Cycle. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{cc1}
\"\"\"

Question: How much is the payments?"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the Platinum Mastercard ending in 8508 Billing Cycle.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

The payments made on the Platinum Mastercard ending in 8508 Billing Cycle are $250.00.


In [ ]:
query = f"""Use the below article on the Platinum Mastercard ending in 8508 Billing Cycle. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{cc1}
\"\"\"

Question: Which is the interest charged?"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the Platinum Mastercard ending in 8508 Billing Cycle.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

Based on the information provided in the article, the interest charged for the Platinum Mastercard ending in 8508 Billing Cycle is $0.00.


In [ ]:
# text copied and pasted from: https://en.wikipedia.org/wiki/Curling_at_the_2022_Winter_Olympics
# I didn't bother to format or clean the text, but GPT will still understand it
# the entire article is too long for gpt-3.5-turbo, so I only included the top few sections

cc2 = """Page 2 of 3
Platinum Card | Platinum Mastercard ending in 8508
Aug 03, 2022 - Sep 02, 2022 | 31 days in Billing Cycle





(Conrours 2:Transactions

8508: Payments, Credits and Adjustments
Trans Date —— Post Date Description
Aug 7 Aug 8 CAPITAL ONE MOBILE PYMTAuthDate 06-Aug
Aug 19 Aug 19 CAPITAL ONE MOBILE PYMTAuthDate 19-Aug

KRYSTAL BRECEDA #8508: Transactions
Trans Date Description
VISIONWORKS INC - GLAS VEGASNYV
AMZN Mktp US*178031AS2Amzn.combillWA

Page 2 of 3
Platinum Card | Platinum Mastercard ending in 8508
Aug 03, 2022 - Sep 02, 2022 | 31 days in Billing Cycle

Amount
$787.14
$25.28



AMZN Mktp US*7K7LM6FA3Amzn.conmvbilIWA
Aug 17 Aug 18 OLD NAVY US 3292LAS VEGASNY
Aug 21 Aug 22 Amazon.com*QG7UL4173AMZN.COM/BILLWA
Aug 28 Aug 29 BURLINGTON STORES 1234LAS VEGASNY
Aug 28 Aug 29 SAM'S TOWN BOWLING CENLAS VEGASNY
KRYSTAL BRECEDA #8508: Total Transactions

Total Transactions for This Period

Trans Date Post Date Description

Total Fees for This Period

Interest Charge on Purchases
Interest Charge on Cash Advances
Interest Charge an Other Balances

Total Interest for This Period

Total Fees charged
Total Interest charged

$10.78
$19.50
$11.35
$77.00
$30.00
$961.05

$961.05




"""

In [ ]:
query = f"""Use the below article on the Platinum Mastercard ending in 8508 Billing Cycle. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{cc2}
\"\"\"

Question: Which is the KRYSTAL BRECEDA #8508: Total Transactions?"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the Platinum Mastercard ending in 8508 Billing Cycle.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

Based on the information provided in the article, the total transactions for the Platinum Mastercard ending in 8508 for the billing cycle are not specified. Therefore, the answer to the question "Which is the KRYSTAL BRECEDA #8508: Total Transactions?" is "I don't know."


In [ ]:
# text copied and pasted from: https://en.wikipedia.org/wiki/Curling_at_the_2022_Winter_Olympics
# I didn't bother to format or clean the text, but GPT will still understand it
# the entire article is too long for gpt-3.5-turbo, so I only included the top few sections

loan1 = """


Oe
scratch* Lendco Persona toa

hitpsifscratch fi Loan number 12345678910

Sammy Scratch Total amount due July 1, 2017

os $316.37

Anytown, USA 12345
After Jul 11, 2017, a $10 late fee wil be charged,

Current payment

Payment overview Account information



Principal $2712.34 Statement date 06/13/2017
Interest $104.03, Loan status Current
Remaining principal $28,002.64
Interest rate 4875%
Maturity date 1111/2026
Principal paid YTD $5,038.26
Total due July 1, 2017 $316.37 Interest paid YTD $1,402.24

Regular monthly payment $316.37
Fees $0.00
Overdue payments $80.00

Activity since last statement (May 13- June 12)

Date Transaction Charge Payment Past payments breakdown



Jun 01 Payment received $316.37 Principal $204.16
Interest $112.21
Fees $0.00

Total $316.37

Additional information



Autopay is on so you're all set! $316.37 will be applied on July 1, 2017 from your account ending in *4234. You can edit or
cancel this payment at any time online at https://scratch.fi, And by paying with Autopay, your interest rate was reduced by 0.25%!

Questions? Visit https://help scratch.fi or call (855) 624-8246, Hours: M-Th 9am - 8pm and F 9am - Spm ET.

Notice for payments by rail: if youre mailing a payment, please print the form below and include it in the envelope. Be sure to write
your loan number in the memo of the check and make your check out to Scratch Services, LLC.

Loan number 1236678910 Send via USPS to: For FedEx, UPS and all others:

Payment due date Jul1, 2017 — Scratch Services, LLC Lockbox Services Box #138654

Total payment due $316.37 PO Box 398654 3440 Walnut Avenue, Bldg 4, Window H
San Francisco, CA 94139-8654 Fremont, CA94538
"""

In [ ]:
query = f"""Use the below article on the scratch* Lendco Personal Loan. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{loan1}
\"\"\"

Question: Which is the Overdue payments?"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the scratch* Lendco Personal Loan.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

The overdue payments amount is $80.00.


In [ ]:
query = f"""Use the below article on the scratch* Lendco Personal Loan. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{loan1}
\"\"\"

Question: Which is the Remaining principal"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the scratch* Lendco Personal Loan.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

The remaining principal is $28,002.64.


In [ ]:
query = f"""Use the below article on the scratch* Lendco Personal Loan. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{loan1}
\"\"\"

Question: How much is due July 1, 2017"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the scratch* Lendco Personal Loan.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

The total amount due on July 1, 2017, is $316.37.
